In [1]:

from math import pi,sqrt
import numpy as np
import circle_fit as cf
import matplotlib.pyplot as plt
import scipy
from scipy import interpolate
from scipy.interpolate import interp1d


In [2]:
#function reads pts file and return a 2-d numpy array of points
def read_points_file(ptsfile_name):
    points_list = []
    with open(ptsfile_name) as f:
        for line in f:
            if "{" in line or "version:" in line or "n_points:" in line:
                continue        # Ignore tags
            if line.isspace():
                continue        # Ignore any empty lines.
            if "}" in line:
                break           # Terminate when we hit the enclosing bracket at the end of the IMAGES section
            x, y = line.split()
            points_list.append([x,y])
    return np.asarray(points_list, dtype=np.float32) 
            

In [3]:
#function writes a 2-d numpy array of points to a newly-created file. 
def create_points_file(ptsfile_name, points):
    f = open(ptsfile_name,'w')
    f.write('version: 1' + '\n')
    f.write('n_points: ' + str(len(points)) + '\n')
    f.write('{\n')
    for i in range(len(points)):
        f.write(str(points[i][0])+' '+ str(points[i][1])+'\n')
    f.write('}')
    f.close()
        

In [4]:
#function returns the shortest distance between point x and the line segment ab.
def shortest_distance_from_point_to_segment(x,a, b):
    u = (b-a) / np.linalg.norm(b-a)
    c= np.dot(u,x-a)
    r= np.linalg.norm(x-a)
    v= np.linalg.norm(b-a)
    l=0
    if c < 0:
        d= r
        l= -1
        z = a
    elif c > v:
        d= np.linalg.norm(x-b)
        l= 1
        z = b
    else: 
        d= sqrt(r*r-c*c) # distance
        z = a + c*u # point
    return d,l,z
    

In [5]:
def mJSW(points):
    #defining mJSW
    acetabulum = list(range(END_POINT_NUMBER,START_POINT_NUMBER_MINUS_1,-1)) #ACETABULAR POINTS - note python does not count initial point so "(84,77,-1)" would be points 78-84 inclusive
    #print (acetabulum),
    fem_head = list(range(START_POINT_NUMBER, END_POINT_NUMBER_PLUS_1})) #"(22,32)" would be points 22-31 inclusive
    #print (fem_head)

    dmin = 10000
    for i in fem_head:
        for j in acetabulum:
            #print(j)
            d, l, z = shortest_distance_from_point_to_segment(points [i], points [j], points [j-1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =i

    for m in acetabulum:
        for n in fem_head :
            #print(i)
            d, l, z = shortest_distance_from_point_to_segment(points [m], points [n], points [n+1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =m
    #print ("the shortest distance d:" ,dmin)
    #print ("the intersect point z: ", zmin)
    #print ("the index of the nearest point to z:" , index_min)
    return   dmin,zmin,index_min

In [ ]:
#Example: how to run the function mJSW on all pts files listed in the file "jsw_6807.txt", 
#writing the outputs to csv file.
pts_files = open('IMAGE_LIST_FILE', 'r+')
f1 = open('OUTPUT_FILE_NAME','w')
for pts_file in pts_files:
    print ("working on file: "+pts_file[:-1])
    points = read_points_file("PATH_TO_POINTS_FILES/"+pts_file[:-1])
    d,z,index = mJSW(points)
    
    f1.write(str(pts_file)+","+str(d)+","+str(index)+","+'\n')
    
f1.close()
pts_files.close()